In [1]:
import numpy as np
import pandas as pd
import json
from glob import glob
import os
from importlib import reload
import utils

In [2]:
sample = 'D2'

wsrc = '../cell_dams/'
nsrc = '../nuclear_mask/'
tsrc = '../translocs/'
ksrc = '../kde/' + sample + os.sep
isrc = '..' + os.sep + 'infected_focus_summer24' + os.sep + sample + os.sep
geosrc =  '..' + os.sep + 'geometry' + os.sep + sample + os.sep

metacell = pd.read_csv(ksrc + sample + '_cells_metadata.csv', index_col='ndimage_cellID')
metaecc = pd.read_csv(ksrc + sample + '_nodule_root_eccentricity.csv', index_col='ndimage_cellID')
metaecc['eccentricity'] = metaecc['eccentricity'].max() - metaecc['eccentricity'].values

metatrans = pd.read_csv(ksrc + sample + '_transcripts_metadata.csv')
transcell = pd.read_csv(ksrc + sample + '_transcells_metadata.csv').set_index('gene')
cell_nuc = pd.read_csv(ksrc + sample + '_nuclei_limits.csv')
cell_nuc.loc[cell_nuc['n_bot'] == 1, 'n_bot'] = 0
cell_nuc.loc[cell_nuc['n_top'] == 61, 'n_top'] = 65

transcriptomes = np.asarray(list(metatrans['gene']))
metatrans = metatrans.set_index('gene')

Cells = utils.get_range_cell_values(isrc + 'infected_cells_ids.csv', metacell, startval=1)
Cells = np.setdiff1d( Cells, metacell[metacell['number_nuclei'] > 1].index)
Genes = utils.get_range_gene_values(isrc + 'genes_to_focus_infection.csv', transcriptomes, startval=0)
invGenes = dict(zip(Genes, range(len(Genes))))
invCells = dict(zip(Cells, range(len(Cells))))
transfocus = transcell.loc[transcriptomes[Genes], Cells.astype(str)]

for t in transcriptomes:
    if not os.path.isdir(geosrc + t):
        os.mkdir(geosrc + t)

In [42]:
for cidx in Cells:
    filename = geosrc + 'peripherality_c{:06d}.csv'.format(cidx)
    peri = pd.read_csv(filename)
    cnames = [ c.split('_-_')[0] for c in peri.columns[1:11] ]
    
    p1 = peri.iloc[:, 1:11]
    p1.columns = cnames
    filename = geosrc + '{}/{}_bins_peripherality_c{:06d}.csv'.format(transcriptomes[Genes[0]], KBINS_NO-1, cidx)
    p1.to_csv(filename, index = False)
    
    p2 = peri.iloc[:, 11:]
    p2.columns = cnames
    filename = geosrc + '{}/{}_bins_peripherality_c{:06d}.csv'.format(transcriptomes[Genes[1]], KBINS_NO-1, cidx)
    p2.to_csv(filename, index = False)

In [43]:
N = 10
for N in [10,15,20,25,30]:
    foo = p1['sum_{}'.format(N)]/p1['mean_{}'.format(N)]
    foo = np.round(foo.values).astype(int)
    print(N, np.sum(foo != peri['count'].values.astype(int)), sep='\t')

10	0
15	0
20	0
25	0
30	0
